In [ ]:
!nvidia-smi

In [ ]:
# !pip install q tensorflow==2.1
# !pip install q keras==2.3.1
# !pip install git+https://github.com/qubvel/segmentation_models

# for str decode error ... run it and restart runtime
# !pip uninstall h5py -y
!pip install h5py==2.10.0

In [ ]:
i = !ls Dataset/images/
l = !ls Dataset/masks/
len(i), len(l)

In [ ]:
l[-5:], i[-5:]

In [ ]:
import cv2
from glob import glob
import os
import numpy as np

# 100 = background
# 101 = road
# 102 = obstacle

l = glob('Dataset/masks/*')
m = cv2.imread(l[2970],0)
m.shape
np.unique(m)

In [ ]:
import os
len(os.listdir('Dataset/images/')), len(os.listdir('Dataset/masks/'))

In [ ]:
import numpy as np
import cv2
import os
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf

def load_dataset(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*")))
    masks = sorted(glob(os.path.join(dataset_path, "masks/*")))

    train_x, test_x, train_y, test_y = train_test_split(images,masks, test_size=0.168, 
                                                        random_state=168, shuffle=True)
    return train_x, train_y, test_x, test_y

train_x, train_y, val_x, val_y = load_dataset('Dataset')
print(len(train_x), len(train_y), len(val_x), len(val_y))

### TFRecord

In [ ]:
r"""Build a TF Record for Cityscapes Semantic Segmentation dataset."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import glob
import io
import json
import os
import numpy as np
import PIL.Image
import tensorflow as tf

def _bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def _int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def _open_file(full_path):
    with tf.io.gfile.GFile(full_path, 'rb') as fid:
        encoded_file = fid.read()
        encoded_file_io = io.BytesIO(encoded_file)
        image = PIL.Image.open(encoded_file_io)
    return image, encoded_file

def create_tf_example(image_path, label_path, image_dir='', is_jpeg=False):
    file_format = 'jpeg' if is_jpeg else 'png'
    full_image_path = os.path.join(image_dir, image_path)
    full_label_path = os.path.join(image_dir, label_path)
    image, encoded_image = _open_file(full_image_path)
    label,encoded_label = _open_file(full_label_path)
    height = image.height
    width = image.width

    if height != label.height or width != label.width:
        raise ValueError('Input and annotated images must have same dims.''verify the matching pair for {}'.format(full_image_path))

    feature_dict = {'image/encoded': _bytes_feature(encoded_image),
                    'image/filename': _bytes_feature(full_image_path.encode('utf8')),
                    'image/format': _bytes_feature(file_format.encode('utf8')),
                    'image/height': _int64_feature(height),
                    'image/width': _int64_feature(width),
                    'image/channels': _int64_feature(3),
                    'image/segmentation/class/encoded': _bytes_feature(encoded_label),
                    'image/segmentation/class/format':_bytes_feature('png'.encode('utf8')),
                    }
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example
            
def _create_tf_record(images, labels, output_path):
    writer = tf.io.TFRecordWriter(output_path)
    for idx, image in enumerate(images):
        if idx % 500 == 0:
            tf.compat.v1.logging.info('On image %d of %d', idx, len(images))
        tf_example = create_tf_example(image, labels[idx], is_jpeg=False)
        writer.write(tf_example.SerializeToString())
    writer.close()
    tf.compat.v1.logging.info('Finished writing!')

_create_tf_record(train_x,train_y,output_path="Dataset/train_cs.record")
_create_tf_record(val_x,val_y,output_path="Dataset/val_cs.record")
# _create_tf_record(val_x,val_y,output_path="Dataset/val_cs.record")

In [ ]:
# raw_dataset = tf.data.TFRecordDataset("Dataset/val_cs.record")
# raw_dataset
# for raw_record in raw_dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)

In [ ]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10